In [5]:
import os
import boto3
import sagemaker
from sagemaker.huggingface import HuggingFacePredictor

region_name = "us-east-1"
aws_access_key = os.getenv("aws_access_key")
aws_secret_access_key = os.getenv("aws_secret_access_key")

def get_sagemaker_response(payload, endpoint_name):
    sagemaker_session = sagemaker.Session(
            boto3.session.Session(
                region_name=region_name,
                aws_access_key_id=aws_access_key,
                aws_secret_access_key=aws_secret_access_key,
            )
        )

    predictor = HuggingFacePredictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sagemaker_session
    )
    
    try:
        output = predictor.predict(payload)
    except Exception:
        print(f"Could not get SageMaker prediction for endpoint {endpoint_name}")
    return output

In [6]:
def get_emotion(text):
    payload = {"inputs": text, "options": {"wait_for_model": True}}
    endpoint_name = "social-signals-emotion-2023-06-03-05-35-02-830"
    output = get_sagemaker_response(payload, endpoint_name)
    
    return output

In [19]:
text = "I am feeling happy, sad, and mad"
output = get_emotion(text)
print(output)

[{'label': 'sadness', 'score': 0.6923155784606934}]


In [8]:
def get_ner(text):
    payload = {"inputs": text, "options": {"wait_for_model": True}}
    endpoint_name = "social-signals-ner-2023-06-04-03-05-56-782"
    output = get_sagemaker_response(payload, endpoint_name)
    
    return output

In [21]:
text = "My "
output = get_ner(text)
print(output)

[]


In [ ]:
CATEGORIES = [
    "Politics",
    "Science",
    "Technology",
    "Sports",
    "Entertainment",
    "Education",
    "Health",
    "Finance",
    "Miscellaneous",
]

def get_classification(text):
    payload = {"inputs": text, "parameters": {"candidate_labels": CATEGORIES}, "options": {"wait_for_model": True}}
    endpoint_name = "social-signals-ner-2023-06-04-03-05-56-782"
    output = get_sagemaker_response(payload, endpoint_name)
    
    return output

In [ ]:
text = "My name is Clara and I live in Berkeley, California."
output = get_ner(text)
print(output)